<a href="https://colab.research.google.com/github/pmy0792/pytorch_tutorial/blob/main/lab_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# For reproducibility
torch.manual_seed(1)

### Training data

In [3]:

x_data=[[1,2],[2,3],[3,1],[4,3],[5,3],[6,2]] # size: (6,2)
y_data=[[0],[0],[0],[1],[1],[1]]

x_train=torch.FloatTensor(x_data)
y_train=torch.FloatTensor(y_data)

print(x_train.shape)
print(y_train.shape)

torch.Size([6, 2])
torch.Size([6, 1])


### Computing the Hypothesis

In [4]:
print('e^1 equals: ',torch.exp(torch.FloatTensor([1])))

e^1 equals:  tensor([2.7183])


In [5]:
W=torch.zeros((2,1),requires_grad=True)
b=torch.zeros(1, requires_grad=True)

In [9]:
hypothesis = 1 / (1+torch.exp(-(x_train.matmul(W)+b)))
# x_train.matmul(W) == torch.matmul(x_Train,W)

In [10]:
print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)
torch.Size([6, 1])


In [11]:
# torch에서 sigmoid 함수 제공
hypothesis = torch.sigmoid(x_train.matmul(W)+b)

In [12]:
print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward0>)
torch.Size([6, 1])


### Computing the Cost Function

In [13]:
losses = -(y_train * torch.log(hypothesis)+
           (1-y_train)*torch.log(1-hypothesis))
print(losses)

tensor([[0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931]], grad_fn=<NegBackward0>)


In [14]:
cost = losses.mean()
print(cost)

tensor(0.6931, grad_fn=<MeanBackward0>)


In [15]:
F.binary_cross_entropy(hypothesis,y_train)

tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward0>)

### Whole Training Procedure

In [20]:
# 모델 초기화
W = torch.zeros((2,1), requires_grad=True)
b= torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer=optim.SGD([W,b],lr=0.5)

nb_epochs=10
for epoch in range(nb_epochs+1):

  # Cost 계산
  hypothesis = torch.sigmoid(x_train.matmul(W)+b)
  cost=F.binary_cross_entropy(hypothesis,y_train)

  # Cost로 H(x) 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  print('Epoch {:4d}/{} Cost: {:.6f}'.format(
      epoch, nb_epochs, cost.item())
  )

Epoch    0/10 Cost: 0.693147
Epoch    1/10 Cost: 0.690280
Epoch    2/10 Cost: 0.627996
Epoch    3/10 Cost: 0.633151
Epoch    4/10 Cost: 0.569236
Epoch    5/10 Cost: 0.566467
Epoch    6/10 Cost: 0.524315
Epoch    7/10 Cost: 0.515848
Epoch    8/10 Cost: 0.491549
Epoch    9/10 Cost: 0.482009
Epoch   10/10 Cost: 0.468286


### Evaluation

In [21]:
hypothesis = torch.sigmoid(x_train.matmul(W)+b) # x_train 대신 x_test 사용
print(hypothesis[:5])

tensor([[0.3020],
        [0.3541],
        [0.7288],
        [0.6844],
        [0.8118]], grad_fn=<SliceBackward0>)


In [32]:
# hypothesis -> binary predictions
prediction = hypothesis >= torch.FloatTensor([0.5])
prediction = prediction.float()
print(prediction[:5])

tensor([[0.],
        [0.],
        [1.],
        [1.],
        [1.]])


In [33]:
correct_prediction=prediction.float()==y_train
print(correct_prediction[:5])

tensor([[ True],
        [ True],
        [False],
        [ True],
        [ True]])


### Higher Implementation with Class

In [54]:
class BinaryClassifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(2,1)
    self.sigmoid = nn.Sigmoid()

  def forward(self,x):
    return self.sigmoid(self.linear(x))

In [55]:
model=BinaryClassifier()

In [58]:
# optimizer 설정
optimizer=optim.SGD(model.parameters(),lr=0.1)

nb_epochs=10
for epoch in range(nb_epochs+1):

  # H(x) 계산
  hypothesis = model(x_train)

  # Cost 계산
  cost=F.binary_cross_entropy(hypothesis,y_train)

  # Cost로 H(x) 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()


  prediction = hypothesis >= torch.FloatTensor([0.5])
  correct_prediction = prediction.float() == y_train
  accuracy = correct_prediction.sum().item() / len(correct_prediction)
  print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format(
      epoch, nb_epochs, cost.item(),accuracy*100)
  )

Epoch    0/10 Cost: 0.583921 Accuracy 50.00%
Epoch    1/10 Cost: 0.540483 Accuracy 66.67%
Epoch    2/10 Cost: 0.516122 Accuracy 66.67%
Epoch    3/10 Cost: 0.503177 Accuracy 83.33%
Epoch    4/10 Cost: 0.496050 Accuracy 83.33%
Epoch    5/10 Cost: 0.491570 Accuracy 83.33%
Epoch    6/10 Cost: 0.488227 Accuracy 83.33%
Epoch    7/10 Cost: 0.485379 Accuracy 83.33%
Epoch    8/10 Cost: 0.482768 Accuracy 83.33%
Epoch    9/10 Cost: 0.480295 Accuracy 83.33%
Epoch   10/10 Cost: 0.477920 Accuracy 83.33%
